In [2]:
import gradio as gr

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [3]:
def initialize_chat_bot(vector_store_dir: str="real_qa_500"):
    db = FAISS.load_local(
        vector_store_dir, 
        OpenAIEmbeddings()
    )
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0
    )
    
    global CHAT_BOT    
    CHAT_BOT = RetrievalQA.from_chain_type(llm,
                                           retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                     search_kwargs={"score_threshold": 0.8}))
    # 返回向量数据库的检索结果
    CHAT_BOT.return_source_documents = True

    return CHAT_BOT

In [4]:
def chat_robot(message, history):
    print(f"[message]{message}")
    print(f"[history]{history}")
    # TODO: 从命令行参数中获取
    enable_chat = True

    ans = CHAT_BOT({"query": message})
    # 如果检索出结果，或者开了大模型聊天模式
    # 返回 RetrievalQA combine_documents_chain 整合的结果
    if ans["source_documents"] or enable_chat:
        print(f"[result]{ans['result']}")
        print(f"[source_documents]{ans['source_documents']}")
        return ans["result"]
    # 否则输出套路话术
    else:
        return "这个你最好去医院看一下"

In [6]:
def launch_gradio():
    demo = gr.ChatInterface(
        fn=chat_robot,
        title="医疗问答",
        # retry_btn=None,
        # undo_btn=None,
        chatbot=gr.Chatbot(height=600),
    )

    demo.launch(share=True, server_name="0.0.0.0")

In [7]:
initialize_chat_bot()
launch_gradio()

Running on local URL:  http://0.0.0.0:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


[message]血尿怎么办
[history][]


d:\Anaconda3\envs\llm\Lib\site-packages\langchain\vectorstores\base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


[result]血尿是指尿液中出现血液，可能是由于尿路感染、结石、肾脏疾病、肿瘤等引起的。如果出现血尿，建议您尽快就医，进行相关检查和诊断，以确定病因并采取相应的治疗措施。请咨询医生以获取专业建议。
[source_documents][]
